In [72]:
using CSV, DataFrames, JuMP, Gurobi, LinearAlgebra, Random

In [73]:
# Constants

nP = 73 # number of people
nT = 210 # number of timeslots in a week
# Organized in 30 minute interals from 9am-12pm, Sunday-Saturday
# 30 intervals/day * 7 days/week
cM = 1e7 # multiplier for the minimal matching objective

print("")

In [74]:
# Load Data

# 0-1 matrix of whether person can mentor/mentee at time T
load_A = Matrix(CSV.read("schedules.csv", DataFrame, header=0))[3:end, 2:end]  # nP x nT
A = zeros(nP, nT)
for i in 1:nP, j in 1:nT
    A[i, j] = parse(Int64, load_A[i, j])
end

@show size(A)

# for these, what if someone is only a mentor or only a mentee? would it be zero?
aR = Matrix(CSV.read("num_sessions.csv", DataFrame, header=0))[:, 2] # nP x 1: max num of session person can mentor
# 
# aE = Matrix(CSV.read("max_menteeing.csv", DataFrame)) # nP x 1: max num of session person can mentee (can we remove this?)
r  = Matrix(CSV.read("rank.csv", DataFrame, header=0))[:, 2]# nP x 1, rank

# Don't we just need one per person? Lessens the amount of data, plus not everyone is a mentor or mentee
p = Matrix(CSV.read("poomsae_pref.csv", DataFrame, header=0))[:, 2]
s = Matrix(CSV.read("sparring_pref.csv", DataFrame, header=0))[:, 2]
# pR = Matrix(CSV.read("mentor_poomsae.csv", DataFrame)) # nP x 1: 0-1 does mentor prefer poomsae 
# pE = Matrix(CSV.read("mentee_poomsae.csv", DataFrame)) # nP x 1: 0-1 does mentee prefer poomsae
# sR = Matrix(CSV.read("mentor_sparring.csv", DataFrame)) # nP x 1: 0-1 does mentor prefer sparring
# sE = Matrix(CSV.read("mentee_sparring.csv")) # nP x 1: 0-1 does mentee prefer sparring

@show aR
@show r
@show p
@show s


print("")

size(A) = (73, 210)
aR = Any[3, 0, 2, 0, 2, 2, 0, 0, 0, 2, 2, 0, 0, 2, 2, 2, 4, 0, 0, 0, 2, 0, 0, 0, 2, 0, 2, 1, 0, 2, 0, 0, 2, 3, 2, 0, 0, 2, 2, 3, 0, 3, 0, 0, 0, 0, 0, 2, 2, 0, 4, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 1, 0, 2, 0, 0, 2, 0, 0, 0, 0, 0, 0]
r = Any[-2, 9, 2, 9, 0, 2, 5, 9, 9, 5, 2, 4, 9, 5, 4, 4, -1, 5, 7, 9, 5, 10, 9, 7, 5, 9, 2, 5, 9, -3, 9, 9, 2, 1, 0, 8, 7, 2, 5, -3, 4, 0, 9, 5, 10, 2, 4, 2, 2, 9, 0, 9, 6, -1, 9, 9, 9, 9, 4, 9, 6, -1, 5, 0, -1, 5, 2, 9, 6, 0, 9, 9, 9]
p = Any[1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1]
s = Any[1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1]


In [75]:
# Create Cost Matrix

alpha = 1
beta  = 1

C = zeros(nP, nP)

# Adjusting to only have one set of data for sparring and poomsae preferences
for i = 1:nP, j = 1:nP
    C[i, j] += alpha * (r[j]-r[i]-2+(r[j] <= 0))
    C[i, j] -= beta  * (p[i]*p[j]+s[i]*s[j]-2*(p[i]*s[i]))
    C[i, j] -= 1
end

# for i = 1:nP, j = 1:nP
#     C[i, j] += alpha * (r[i]-r[j]-2+(r[j] >= 9))
#     C[i, j] -= beta  * (pR[i]*pE[j]+sR[i]*sE[j]-pR[i]*sR[i]-pE[j]*sE[j])
#     C[i, j] -= 1
# end


In [76]:
# Rank Constraint Matrix
# Adjust to be negative hehe

R = zeros(nP, nP, nT)

for i = 1:nP, j = 1:nP, T = 1:nT
    if r[i] <= r[j]-2 || (r[i] <= r[j]-1 && r[j] <= 0)
        R[i, j, T] = 1
    end
end

R = convert(Array{Int64}, R);

In [77]:
m = Model(Gurobi.Optimizer)
# set_optimizer_attribute(m, "OutputFlag", 0)

print("")

Set parameter Username
Academic license - for non-commercial use only - expires 2023-12-03


In [ ]:
@variable(m, 0 <= x[1:nP, 1:nP, 1:nT] <= 1, Int)
@variable(m, 0 <= t[1:nP] <= 1, Int)

print("")

In [ ]:
# Rank Constraint (capacity constraint)
@constraint(m, x .<= R);

In [ ]:
# Maximum Constraints
for P = 1:nP
    @constraint(m, sum(x[i, P, T] for i=1:nP, T = 1:nT) <= 1);
    @constraint(m, sum(x[P, i, T] for i=1:nP, T = 1:nT) <= aR[P]); # replaced with ones
end

In [ ]:
# Availability Constraints
for i = 1:nP, j = i+1:nP, T=1:nT
    @constraint(m, x[i,j,T] <= A[i,T]*A[j,T]);
end

In [ ]:
# # THIS ONE TAKES TOO LONG

# # Single Mentor Constraint
# for i = 1:nP, j = i+1:nP, k = 1:nP, T=1:nT
#     @constraint(m, x[i,k,T]+x[j,k,T] <= 1);
# end

In [ ]:
# Making t work
for P = 1:nP
    @constraint(m, sum(x[i,P,T] for i=1:nP, T=1:nT) >= t[P])
    @constraint(m, 1*sum(x[i,P,T] for i=1:nP, T=1:nT) <= t[P]);  # make constraint integral
end

print("")

In [ ]:
@objective(m, Min, cM*sum((1-t[P]) for P=1:nP) + sum(x[i,j,T]*C[i,j] for i=1:nP, j=1:nP, T=1:nT))
print("")

In [ ]:
optimize!(m)

objective_value(m)

In [ ]:
count = 0
x_sol = value.(x)
for i in 1:nP, j in 1:nP, k in 1:nT
    if x_sol[i, j, k] != 0
        count += 1
    end
end
@show count

In [ ]:
names = Matrix(CSV.read("num_sessions.csv", DataFrame, header=0))[:, 1]
# @show names

# count = 0
# x_sol = value.(x)
for i in 1:nP, j in 1:nP, k in 1:nT
    if x_sol[i, j, k] != 0
        print("$(names[i]), $(names[j]), $(x_sol[i, j, k])\n")
    end
end
# @show count